In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas_bokeh
pandas_bokeh.output_notebook()

import scikitplot as skplt
import seaborn as sns
sns.set()

from sklearn import metrics
from sklearn.model_selection import cross_validate

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15

Loading BokehJS ...

In [2]:
#importing datasets

cab_df = pd.read_csv("Cab_Data.csv")
cab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  int64  
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 19.2+ MB


In [3]:
cab_df.isna().sum()

Transaction ID    0
Date of Travel    0
Company           0
City              0
KM Travelled      0
Price Charged     0
Cost of Trip      0
dtype: int64

There are no missing values in this dataset.

In [4]:
cab_df.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


The `Date of Travel` is of serial number format. Extracting data information from this column.

In [5]:
from datetime import datetime
import xlrd

In [6]:
%%time
dates = [
    pd.to_datetime(xlrd.xldate_as_datetime(date, 0)) for date in cab_df['Date of Travel']
] 

Wall time: 25.2 s


In [7]:
cab_df['Travel_Date'] = dates

In [8]:
cab_df.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip,Travel_Date
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635,2016-01-08
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854,2016-01-06
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632,2016-01-02
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602,2016-01-07
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776,2016-01-03


In [9]:
#Dropping 'Date of Travel' column
cab_df.drop('Date of Travel', axis = 1, inplace = True)

In [10]:
cab_df.head()

,Transaction ID,Company,City,KM Travelled,Price Charged,Cost of Trip,Travel_Date
0,10000011,Pink Cab,ATLANTA GA,30.45,370.95,313.635,2016-01-08
1,10000012,Pink Cab,ATLANTA GA,28.62,358.52,334.854,2016-01-06
2,10000013,Pink Cab,ATLANTA GA,9.04,125.20,97.632,2016-01-02
3,10000014,Pink Cab,ATLANTA GA,33.17,377.40,351.602,2016-01-07
4,10000015,Pink Cab,ATLANTA GA,8.73,114.62,97.776,2016-01-03


In [11]:
#Import city dataset
city_df = pd.read_csv('City.csv')
city_df

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"
5,ORANGE COUNTY,"1,030,185","12,994"
6,SAN DIEGO CA,"959,307","69,995"
7,PHOENIX AZ,"943,999","6,133"
8,DALLAS TX,"942,908","22,157"
9,ATLANTA GA,"814,885","24,701"


In [12]:
#checking if Cities in city_df are in cab_df

np.setdiff1d(city_df.City, cab_df.City)

array(['SAN FRANCISCO CA'], dtype=object)

Travel information regarding travelers from San Francisco are not present in the cab_df dataset. Therefore, when both cab_df and city_df will be inner-joined based on the city column, the row containing San Francisco city in city_df will be dropped.

In [13]:
df1 = pd.merge(cab_df, city_df, on = 'City')

In [14]:
# importing Transaction ID dataset

trans_df = pd.read_csv('Transaction_ID.csv')
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440098 entries, 0 to 440097
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Transaction ID  440098 non-null  int64 
 1   Customer ID     440098 non-null  int64 
 2   Payment_Mode    440098 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.1+ MB


In [15]:
trans_df.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [16]:
#Checking for Transaction ID's not in df1 dataset

len(np.setdiff1d(trans_df['Transaction ID'], df1['Transaction ID']))

80706

There are about 80,706 transaction ID's not in the df1 dataset. These will be dropped when joining trans_df with the df1 dataset

In [17]:
df2 = pd.merge(df1, trans_df, on = 'Transaction ID')

In [18]:
# importing customer ID df

cust_df = pd.read_csv('Customer_ID.csv')
cust_df.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [19]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         49171 non-null  int64 
 1   Gender              49171 non-null  object
 2   Age                 49171 non-null  int64 
 3   Income (USD/Month)  49171 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [20]:
# Checking for Customer ID's not in df2 dataset

len(np.setdiff1d(cust_df['Customer ID'], df2['Customer ID']))

3023

Around 3023 Customer ID's are not in the df2 dataset. These will be dropped when cust_df will be joined with the df2 dataset. 

In [21]:
full_df = pd.merge(df2, cust_df, on = 'Customer ID')
full_df.head()

,Transaction ID,Company,City,KM Travelled,Price Charged,Cost of Trip,Travel_Date,Population,Users,Customer ID,Payment_Mode,Gender,Age,Income (USD/Month)
0,10000011,Pink Cab,ATLANTA GA,30.45,370.95,313.6350,2016-01-08,"814,885","24,701",29290,Card,Male,28,10813
1,10351127,Yellow Cab,ATLANTA GA,26.19,598.70,317.4228,2018-07-21,"814,885","24,701",29290,Cash,Male,28,10813
2,10412921,Yellow Cab,ATLANTA GA,42.55,792.05,597.4020,2018-11-23,"814,885","24,701",29290,Card,Male,28,10813
3,10000012,Pink Cab,ATLANTA GA,28.62,358.52,334.8540,2016-01-06,"814,885","24,701",27703,Card,Male,27,9237
4,10320494,Yellow Cab,ATLANTA GA,36.38,721.10,467.1192,2018-04-21,"814,885","24,701",27703,Card,Male,27,9237


In [22]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359392 entries, 0 to 359391
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction ID      359392 non-null  int64         
 1   Company             359392 non-null  object        
 2   City                359392 non-null  object        
 3   KM Travelled        359392 non-null  float64       
 4   Price Charged       359392 non-null  float64       
 5   Cost of Trip        359392 non-null  float64       
 6   Travel_Date         359392 non-null  datetime64[ns]
 7   Population          359392 non-null  object        
 8   Users               359392 non-null  object        
 9   Customer ID         359392 non-null  int64         
 10  Payment_Mode        359392 non-null  object        
 11  Gender              359392 non-null  object        
 12  Age                 359392 non-null  int64         
 13  Income (USD/Month)  359392 no

In [23]:
#Checking for NA's

full_df.isna().sum()

Transaction ID        0
Company               0
City                  0
KM Travelled          0
Price Charged         0
Cost of Trip          0
Travel_Date           0
Population            0
Users                 0
Customer ID           0
Payment_Mode          0
Gender                0
Age                   0
Income (USD/Month)    0
dtype: int64

We have joined all datasets and created a master datasets comprising of columns from all of the datasets.

Next, splitting the `City` column to city and state columns. Fortunatly, there already exists a forum on the internet with a list of all US States abbreviations in a list format.https://snipplr.com/view/50728/list-of-us-state-abbreviations 

In [24]:
%%time
US_States = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

state = []
city = []

for i in range(len(full_df)):
    if full_df.City[i].split()[~0] in US_States:
        city.append(full_df.City[i].split(f' {full_df.City[i].split()[~0]}')[0])
        state.append(full_df.City[i].split()[~0])
    else:
        city.append(full_df.City[i])
        state.append(np.nan)

Wall time: 12.4 s


In [25]:
full_df['City'] = city
full_df['State'] = state

full_df.head()

,Transaction ID,Company,City,KM Travelled,Price Charged,Cost of Trip,Travel_Date,Population,Users,Customer ID,Payment_Mode,Gender,Age,Income (USD/Month),State
0,10000011,Pink Cab,ATLANTA,30.45,370.95,313.6350,2016-01-08,"814,885","24,701",29290,Card,Male,28,10813,GA
1,10351127,Yellow Cab,ATLANTA,26.19,598.70,317.4228,2018-07-21,"814,885","24,701",29290,Cash,Male,28,10813,GA
2,10412921,Yellow Cab,ATLANTA,42.55,792.05,597.4020,2018-11-23,"814,885","24,701",29290,Card,Male,28,10813,GA
3,10000012,Pink Cab,ATLANTA,28.62,358.52,334.8540,2016-01-06,"814,885","24,701",27703,Card,Male,27,9237,GA
4,10320494,Yellow Cab,ATLANTA,36.38,721.10,467.1192,2018-04-21,"814,885","24,701",27703,Card,Male,27,9237,GA


In [26]:
full_df.City.unique()
print()
full_df.State.unique()

array(['GA', 'TX', 'MA', 'IL', 'CO', 'CA', 'FL', 'TN', 'NY', nan, 'AZ',
       'PA', 'WA', 'DC'], dtype=object)

In [27]:
full_df.isna().sum()

Transaction ID            0
Company                   0
City                      0
KM Travelled              0
Price Charged             0
Cost of Trip              0
Travel_Date               0
Population                0
Users                     0
Customer ID               0
Payment_Mode              0
Gender                    0
Age                       0
Income (USD/Month)        0
State                 12501
dtype: int64

There are about 12501 observations with missing states columns. Checking which cities have missing states. 

In [28]:
full_df[full_df.State.isna()]['City'].unique()

array(['ORANGE COUNTY', 'SILICON VALLEY'], dtype=object)

Since both Orange County and Silicon Valley are located within the State of California, I will impute the missing values in the `State`column belonging to these cities as 'CA' (California)

In [29]:
full_df.fillna('CA', inplace = True)

In [30]:
full_df.isna().sum()

Transaction ID        0
Company               0
City                  0
KM Travelled          0
Price Charged         0
Cost of Trip          0
Travel_Date           0
Population            0
Users                 0
Customer ID           0
Payment_Mode          0
Gender                0
Age                   0
Income (USD/Month)    0
State                 0
dtype: int64

##### Now the dataset is complete and ready for EDA

In [31]:
full_df.to_csv('full_df.csv', index = False)